# TomoBase Tutorial

This notebook is designed to show the basic processes of how to use the tomobase library for running tomographic processes. To do so each cell in this library outlines how to perform a procedure with the library:

Table of Contents:
[t](#data-imports)


## Introduction

## 

In [1]:
import tomobase
from tomobase import data
from tomobase.processes import alignments
import ncempy.io as nio
import numpy as np 
import astra


2025-03-17 13:32:45,148 - ERROR - hyperspy module not found.


In [ ]:
from skimage.util import random_noise
import matplotlib.pyplot as plt
from tomobase import phantoms 

vol = tomobase.phantoms.nanocage()
scheme  =  tomobase.tiltschemes.Binary(-64, 64, 8)
angles = np.array([scheme.get_angle() for i in range(65)])
sino = tomobase.processes.project(vol, angles)
sino.data = np.random.poisson(sino.data)

plt.hist(sino.data.ravel(), bins=100, fc='k', ec='k')
plt.show()

plt.imshow(sino.data[0,:, :])

[-64.   -48.94 -33.88 -18.82  -3.76  11.29  26.35  41.41  56.47  48.94
  33.88  18.82   3.76 -11.29 -26.35 -41.41 -56.47 -60.24 -45.18 -30.12
 -15.06   0.    15.06  30.12  45.18  60.24  52.71  37.65  22.59   7.53
  -7.53 -22.59 -37.65 -52.71 -62.12 -47.06 -32.   -16.94  -1.88  13.18
  28.24  43.29  58.35  50.82  35.76  20.71   5.65  -9.41 -24.47 -39.53
 -54.59 -58.35 -43.29 -28.24 -13.18   1.88  16.94  32.    47.06  62.12
  54.59  39.53  24.47   9.41  -5.65]


TypeError: poisson() got multiple values for keyword argument 'lam'

In [ ]:
sino = alignments.align_sinogram_center_of_mass(sino2)
sino = alignments.align_sinogram_xcorr(sino)
sino = alignments.background_subtract_median(sino)
sino.show()

In [4]:
sino = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpinc_aligned.mrc')
obj = alignments.MaskBackgroundManual(sino)


{'dimensions': array([512, 512,  65], dtype=int32), 'MRCtype': np.int64(2), 'compressor': None, 'dtype': '<f4', 'pixelsize': array([3.87999993e-10, 3.87999993e-10, 3.88000005e-10]), 'pixelunits': '\\AA', 'minImage': np.float32(-2171.297), 'maxImage': np.float32(19445.91), 'meanImage': np.float32(8637.307), 'extendedBytes': 1712, 'metaId': b'json', 'voltage': 0.0, 'C3': 0.0, 'gain': 1.0, 'packedBytes': (0,), 'angles': [-64.0, -62.0, -60.0, -58.0, -56.0, -54.0, -52.0, -50.0, -48.0, -46.0, -44.0, -42.0, -40.0, -38.0, -36.0, -34.0, -32.0, -30.0, -28.0, -26.0, -24.0, -22.0, -20.0, -18.0, -16.0, -14.0, -12.0, -10.0, -8.0, -6.0, -4.0, -2.0, 0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 17.99, 19.99, 21.99, 24.0, 26.0, 28.0, 30.0, 32.0, 34.0, 36.0, 37.99, 40.0, 42.0, 44.0, 46.0, 47.99, 50.0, 52.0, 54.0, 56.0, 58.0, 60.0, 62.0, 64.0], 'times': [11.866396768297136, 178.18059555260749, 240.6228149361412, 274.26372651760715, 314.84597075360733, 358.1849991929932, 386.8964295558288, 417.24443276

In [8]:
from skimage.filters import threshold_otsu
from scipy.ndimage import  binary_dilation
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import tomobase.processes.alignments

sino = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpinc_aligned.mrc')
threshold = threshold_otsu(sino.data)
mask = sino.data < threshold
mask = binary_dilation(mask, iterations=4)

sino_inc = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpinc_aligned.mrc')
sino_bin =  tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpbin_aligned.mrc')
sino_grs = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpgrs_aligned.mrc')

sinos = [sino_inc, sino_bin, sino_grs]
for sino in sinos:
    threshold = threshold_otsu(sino.data)
    mask = sino.data < threshold
    mask = binary_dilation(mask, iterations=4)
    sino.data[~mask] = 0
    sino = tomobase.processes.alignments.normalize(sino)
    rec = tomobase.processes.reconstruct_weighted_sirt(sino, iterations=100, weighted=True)
    sino = tomobase.processes.alignments.weight_by_angle(sino)
    
    projections = tomobase.processes.project(rec, sino.angles)
    mae = np.mean(np.abs(projections.data - sino.data))
    ssim = structural_similarity(projections.data, sino.data, data_range=(sino.data.max() - sino.data.min()))
    print(mae, ssim)
    



{'dimensions': array([512, 512,  65], dtype=int32), 'MRCtype': np.int64(2), 'compressor': None, 'dtype': '<f4', 'pixelsize': array([3.87999993e-10, 3.87999993e-10, 3.88000005e-10]), 'pixelunits': '\\AA', 'minImage': np.float32(-2171.297), 'maxImage': np.float32(19445.91), 'meanImage': np.float32(8637.307), 'extendedBytes': 1712, 'metaId': b'json', 'voltage': 0.0, 'C3': 0.0, 'gain': 1.0, 'packedBytes': (0,), 'angles': [-64.0, -62.0, -60.0, -58.0, -56.0, -54.0, -52.0, -50.0, -48.0, -46.0, -44.0, -42.0, -40.0, -38.0, -36.0, -34.0, -32.0, -30.0, -28.0, -26.0, -24.0, -22.0, -20.0, -18.0, -16.0, -14.0, -12.0, -10.0, -8.0, -6.0, -4.0, -2.0, 0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 17.99, 19.99, 21.99, 24.0, 26.0, 28.0, 30.0, 32.0, 34.0, 36.0, 37.99, 40.0, 42.0, 44.0, 46.0, 47.99, 50.0, 52.0, 54.0, 56.0, 58.0, 60.0, 62.0, 64.0], 'times': [11.866396768297136, 178.18059555260749, 240.6228149361412, 274.26372651760715, 314.84597075360733, 358.1849991929932, 386.8964295558288, 417.24443276

In [ ]:
from tomobase import processes
import tomobase.processes.alignments

sino = tomobase.processes.alignments.align_tilt_axis_shift(sino)

In [ ]:
sino2 = obj.sino  
sino.show()

In [ ]:
sino = processes.alignments.bin(sino)
rec = processes.reconstruct(sino, 'sirt', iterations=100)
rec.to_file('AuNP-Incremental.rec')